Let's explore raw dataset to understand question structure, categories and data quality.

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

BASE_DIR = Path.cwd().parent
PATH = BASE_DIR / "data" / "raw" / "celebrities.csv"

df = pd.read_csv(PATH)

df.head()

,Questions,Correct,A,B,C,D
0,One of Johnny Depps earliest film roles was as...,Platoon,The Color of Money,Platoon,Top Gun,Aliens
1,"Which of these Tim Burton films, starring John...",Edward Scissorhands,Charlie and the Chocolate Factory,Ed Wood,Sleepy Hollow,Edward Scissorhands
2,"This Scottish author, portrayed by Johnny Depp...",James Barrie,John Dunne,George Eliot,Joseph Grimm,James Barrie
3,"In the 2001 movie Blow, Johny Depps role is ba...",True,True,False,NaN,NaN
4,What is the first name of Johnny Depps charact...,Mort,Tom,Alan,Lou,Mort


In [2]:
# Inspect columns, head, missing values

In [3]:
# Analyze categories and difficulties

In [4]:
# Decide final question structure for the game

For the chaser game, each question will be normalized into the following structure:

{
    "id": str,
    "question": str,
    "options": {
        "A": str,
        "B": str,
        "C": str,
        "D": str
    },

    "correct_option": "A" | "B" | "C" | "D",
}